# Automatyczne generowanie książki kodów za pomocą `jinja2`

Kiedy mamy jakieś dane bardzo dobrze byłoby wygenerować odpowiednią "książkę kodów (*code book*), w której mielibyśmy podstawowe informacje dotyczące zmiennych i krótki opis czego te zmienne dotyczą. Oczywiście można zrobić to ręcznie i po prostu napisać odpowiedni dokument, korzystając z Pythona możemy to zadanie nieco zautomatyzować i ułatwić. W R istnieją specjalne pakiety do tego (np. `dataMaid`), w Pythonie nie ma jednak takich. Możemy jednak dość łatwo zreplikować część funkcjonalności w Pythonie. 

W przykładzie posłużymy się wygenerowanymi przeze mnie przykładowymi danymi, które mogłyby pochodzić z eksperymentu sprawdzającego efekt Stroopa dla dwóch języków - pierwszego (polski) i drugiego (angielski).

In [125]:
# Normalnie nie będziemy korzystać z tych pakietów tylko z lepszych wersji, ale tutaj użyjemy ich sobie dla kolorytu
import csv
from statistics import mean, median, stdev
from collections import defaultdict
from jinja2 import Template
from IPython.display import Markdown

Najpierw musimy wczytać dane. W dalszej części kursu nauczymy się jak robić to za pomocą pakietu `pandas`, tutaj jednak użyjemy modułu `csv` z biblioteki standardowej.

In [40]:
with open('../07_Wizualizacje_danych_w_Pythonie/bi-stroop_sex.csv') as f: # otwieramy plik
    data = csv.DictReader(f) # tworzymy obiekt wczytujący plik
    data = list(data) # konwertujemy do postaci listy
for record in data:
    record['rt'] = float(record['rt']) # `rt` oznacza czas reakcji i chcemy aby miał typ `float`

Nasze dane wyglądają tak jak poniżej. Jest to po prostu lista słowników (obiektów `OrderedDict` z modułu `collections`). Poniżej wyświetlam pierwsze 3 obserwacje.

In [41]:
data[:3]

[OrderedDict([('participant', '1'),
              ('condition', 'congruent'),
              ('rt', 192.7897915545731),
              ('lang', 'pol'),
              ('sex', 'K')]),
 OrderedDict([('participant', '1'),
              ('condition', 'incongruent'),
              ('rt', 338.3846854098748),
              ('lang', 'pol'),
              ('sex', 'K')]),
 OrderedDict([('participant', '1'),
              ('condition', 'congruent'),
              ('rt', 129.85952872808804),
              ('lang', 'pol'),
              ('sex', 'K')])]

To co teraz chcemy zrobić, to skonstruować dla każdej kolumny w naszych danych zbiór metadanych. Jeśli zmienna jest kategorialna, to chcielibyśmy zobaczyć jakie występują wartości, jaka jest najczęstsza i jaka jest najrzadsza. Jeśli jest to zmienna liczbowa, to chcemy obliczyć średnią, medianę oraz odchylenie standardowe.

In [136]:
metadata = {}
gather = defaultdict(list) # defaultdict tworzy słownik, gdzie domyślną wartością dla każdego klucza jest `list`

# Iterujemy po wszystkich obserwacjach i zbieramy wartości do odpowiednich list
for record in data:
    for k, v in record.items():
        gather[k].append(v)
        
# Kiedy mamy zebrane wartości możemy stworzyć słowniki z metadanymi dla każdej kolumny

for k,v in gather.items():
    try:
        metadata[k] = {
            'type' : 'numeric',
            'mean' : mean(v),
            'std' : stdev(v),
            'median' : median(v)
        }
    except:
        metadata[k] = {
            'type' : 'factor',
            'values' : sorted(list(set(v))),
            'most' : max(set(v), key=v.count),
            'least' : min(set(v), key=v.count)
        }


Chcielibyśmy również opisać nasze zmienne. W tym celu do każdego słownika możemy dodać klucz `description`, gdzie umieścimy prosty opis w języku polskim, który wyjaśni co to jest za zmienna.

In [137]:
metadata['participant']['description'] = 'Numer porządkowy uczestnika'
metadata['rt']['description'] = 'Czas reakcji w próbie wyrażony w milisekundach'
metadata['sex']['description'] = "Płeć, K=Kobieta, M=Mężczyzna"

Poniżej jest krótki szablon w języku szablonów `jinja2` (w składni Markdowna). Po przejściu przez wstęp do `jinja2` wszystko tutaj powinno być jasne.

In [152]:
template_string = '''# Książka kodowa dla pliku {{ filename }}
Autor: {{ author }}

Opis: {{ description }}

Liczba obserwacji: {{ n_obs }}
{% for variable, data in metadata.items() %}
## Nazwa kolumny: "{{ variable }}"
+ **typ**: {{ data.type }}\
{% if data.description %}
+ **opis** {{ data.description }}\
{% endif %}
{% if data.type == "factor" %}
+ **wartości**: {{ data['values'] }}
+ **najczęściej występująca wartość**: {{ data.most }}
+ **najrzadziej występująca wartość**: {{ data.most }}
{% else %}
+ **średnia**: {{ data.mean }}
+ **mediana**: {{ data.median }}
+ **odchylenie standardowe**: {{ data.std }}
{% endif %}
{% endfor %}
'''

Teraz możemy wygenerować naszą książkę kodową. Oprócz metadanych dotyczących kolumn przekażemy do szablonu jeszcze kilka informacji - nazwę pliku, opis zbioru danych, liczbę obserwacji oraz autora danych (w tym wypadku mnie).

In [153]:
template = Template(template_string)
codebook = template.render(metadata = metadata,
                          filename = 'bi-stroop_sex.csv',
                          description = 'Wygenerowane za pomocą skryptu dane przypominające dane z eksperymentu Stoopa',
                          n_obs = len(data),
                          author = 'Bartosz Maćkiewicz')

Teraz możemy zobaczyć jak wygląda nasza książka kodowa. Całkiem nieźle!

In [154]:
print(codebook)

# Książka kodowa dla pliku bi-stroop_sex.csv
Autor: Bartosz Maćkiewicz

Opis: Wygenerowane za pomocą skryptu dane przypominające dane z eksperymentu Stoopa

Liczba obserwacji: 4000

## Nazwa kolumny: "participant"
+ **typ**: factor
+ **opis** Numer porządkowy uczestnika

+ **wartości**: ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '5', '6', '7', '8', '9']
+ **najczęściej występująca wartość**: 22
+ **najrzadziej występująca wartość**: 22


## Nazwa kolumny: "condition"
+ **typ**: factor

+ **wartości**: ['congruent', 'incongruent']
+ **najczęściej występująca wartość**: incongruent
+ **najrzadziej występująca wartość**: incongruent


## Nazwa kolumny: "rt"
+ **typ**: numeric
+ **opis** Czas reakcji w próbie wyrażony w milisekundach

+ **średnia**: 396.51166261475345
+ **mediana**: 364.20721469939906
+ **odchylenie standardowe*

Naprawde ładnie wygląda jednak dopiero wówczas, gdy wyrenderujemy ją jako sformatowany tekst.

In [155]:
Markdown(codebook)

# Książka kodowa dla pliku bi-stroop_sex.csv
Autor: Bartosz Maćkiewicz

Opis: Wygenerowane za pomocą skryptu dane przypominające dane z eksperymentu Stoopa

Liczba obserwacji: 4000

## Nazwa kolumny: "participant"
+ **typ**: factor
+ **opis** Numer porządkowy uczestnika

+ **wartości**: ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '5', '6', '7', '8', '9']
+ **najczęściej występująca wartość**: 22
+ **najrzadziej występująca wartość**: 22


## Nazwa kolumny: "condition"
+ **typ**: factor

+ **wartości**: ['congruent', 'incongruent']
+ **najczęściej występująca wartość**: incongruent
+ **najrzadziej występująca wartość**: incongruent


## Nazwa kolumny: "rt"
+ **typ**: numeric
+ **opis** Czas reakcji w próbie wyrażony w milisekundach

+ **średnia**: 396.51166261475345
+ **mediana**: 364.20721469939906
+ **odchylenie standardowe**: 163.40659244477507


## Nazwa kolumny: "lang"
+ **typ**: factor

+ **wartości**: ['eng', 'pol']
+ **najczęściej występująca wartość**: eng
+ **najrzadziej występująca wartość**: eng


## Nazwa kolumny: "sex"
+ **typ**: factor
+ **opis** Płeć, K=Kobieta, M=Mężczyzna

+ **wartości**: ['K', 'M']
+ **najczęściej występująca wartość**: K
+ **najrzadziej występująca wartość**: K

